# Lindhardt susceptibility $\chi_0$ for a non interacting electron gas


The Lindhard function is defined by the following equation where, for short cut, $q=(\mathbf{q},iq_n)$ with $iq_n$ bosonic Matsubara frequencies and $k=(\mathbf{k},ik_n)$ with $ik_n$ fermionic Matsubara frequencies

\begin{equation}
\chi_0(q)=-2\frac{T}{N}\sum_{k} G_0(k)G_0(k+q) 
\end{equation}

This has the form of a convolution. So, to speed up the calculation, the evaluation is done by going to imaginary time and lattice positions for the Green's function with fast Fourier transforms (FFT) and performing the product

\begin{equation}
\chi_0(\mathbf{r},\tau)=G_0(\mathbf{r},\tau)G_0(-\mathbf{r},-\tau)
\end{equation}

One finally goes back to momentum and Matsubara frequency space by fast Fourier transforms on $\chi(\mathbf{r},\tau)$

NOTE: One also often denotes the Lindhard function by $\Pi$, the polarisation. This is the convention in the code.

    How many operations are saved by using this trick of fast Fourier transforms?
    At what wavevector is the maximum of the Lindhard function at zero frequency?
    How is the postiion of that maximum related to the Fermi surface?
    Can you plot the function at different temperatures and understand what happens physically?
    What happens if you increase U?
    


The two-particle response of the non-interacting system is given by the Lindhardt susceptibility $\chi_0$. Diagrammatically it is a bubble diagram of two single-particle Green's function, that is most efficiently evaluated in imaginary time.

$$ \chi_0(\tau, r) \equiv 2 g_0(\tau, r) g_0(\beta - \tau, -r) $$

after evaluating the imaginary time bubble we transform $\chi_0$ back to momentum and frequency space, i.e.


$$ \chi_0(i\omega_n, r) \equiv \mathcal{F}_{\tau \rightarrow i\omega_n} \big\{ \chi_0(\tau, r) \big\}$$

$$ \chi_0(i\omega_n, k) \equiv \mathcal{F}_{r \rightarrow k} \big\{ \chi_0(i\omega_n, r) \big\}$$


## The bubble calculation in C++

While it is possible to do this computation in pure Python, it is relatively slow, due to the double loop in $r$ and $\tau$. 

Here we illustrate the C++ layer of the TRIQS library, using the TRIQS/cpp2py tool to wrap Python and C++
in a simple case.

The function bubble below computes the Lindhardt function from Eq (XX).
The following cell compiles the function "bubble" in C++, exposes ("wraps") it to Python and loads it
in the current scope.


In [ ]:
%reload_ext cpp2py.magic

May take a few seconds to compile ...

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_type = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;
using g_r_t_type = gf<cartesian_product<cyclic_lattice, imtime>, scalar_valued>;

g_k_w_type bubble(g_k_w_type g0) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    auto grt = make_gf_from_fourier<0,1>(g0);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mr, mt] = grt.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (r, tau) with this new mesh.
    auto chi0 = g_r_t_type{{mr, mtb}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[r, t] = grt(-r, beta - t) * grt(r, t); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

## Static Lindhardt susceptibility $\chi_0(k, \omega=0)$

The square lattice with only nearest-neighbour hopping $t$ has a property of "perfect nesting", meaning that large parts of the fermi surface are wrapped on-to each other by a single momentum transfer $k$. Go back to the fermi-surface polot of $\epsilon_k$ and determine this peculiar momentum vector.

The "perfect nesting" greatly enhances the particle-hole susceptibility of the system and the Static Lindhardt susceptibility $\chi_0(\omega=0, k)$ has a dominant peak at this momentum.

TODO : 

- Grab the Gf from HDF5 (to pass from a notebook to the next and keep the notebook simple !)

- Call Lindhard to obtain chi(q, omega)

- Plot chi(k, 0) with a slice
  and other plots.
  
- We save the function in HDF5 for later reuse.  
  
- Physics discussion ?

## Load previous G0

In [ ]:
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, Idx
from pytriqs.archive import HDFArchive

with HDFArchive("tpsc.h5",'r') as R:
    g0 = R['g0_kw']

## Compute the bubble

Maybe ask to plot it vs Matsubara frequencies to see that at those temperatures it is essentially
non-zero only very close to 0.

In [ ]:
chi0_kw = bubble(g0)
print chi0_kw

## Save the bubble for later use

In [ ]:
from pytriqs.archive import HDFArchive
with HDFArchive("tpsc.h5") as R:
    R['chi0_kw'] = chi0_kw

## Plot the suscptibility

In [ ]:
%matplotlib inline
from pytriqs.plot.mpl_interface import plt
import numpy as np

nk = g0.mesh[0].linear_dims[0]+1
k = np.linspace(0, 2*np.pi, nk, endpoint=True)
kx, ky = np.meshgrid(k, k)

chi = lambda kx, ky: chi0_kw([kx,ky,0],0).real

# 2d plot
plt.figure(figsize=(3.25*2, 5))
plt.title('Static Lindhardt susceptibility $\chi_0(k, \omega=0)$')
plt.pcolor(kx, ky, np.vectorize(chi)(kx,ky))
plt.xlim(0, 2*np.pi)
plt.ylim(0, 2*np.pi)
plt.xticks([0, np.pi, 2*np.pi],[r"0",r"$\pi$",r"$2\pi$"])    
plt.yticks([0, np.pi, 2*np.pi],[r"0",r"$\pi$",r"$2\pi$"])
plt.colorbar()
plt.tight_layout()
plt.xlabel(r'$k_x$')
plt.ylabel(r'$k_y$')


# Here is a 3d plot. With "matplotlib inline" one cannot interact
from mpl_toolkits.mplot3d import axes3d, Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(kx, ky, np.vectorize(chi)(kx,ky), cmap="jet")
ax.set_xlabel(r"$k_x$")
ax.set_ylabel(r"$k_y$")
ax.set_zlabel(r"$\chi^{0}$")

## Plot along a given path in k (TODO)

In [ ]:
# CLEAN THIS TO MAKE IT MORE ELEGANT!

kx = np.concatenate([np.linspace(0, np.pi, 100), np.ones(100) * np.pi, np.linspace(np.pi, 0, 100)])
ky = np.concatenate([np.linspace(0, np.pi, 100), np.linspace(np.pi, 0, 100), np.zeros(100)])

plt.figure(figsize=(6,4))
plt.title('Static Lindhardt susceptibility $\chi_0(k,\omega=0)$')
plt.plot(np.arange(300), np.vectorize(chi)(kx,ky))

plt.grid();
plt.axes().set_xticks([0,100,200,300])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$']);
plt.ylabel(r'$\chi_0(k,\omega=0)$')
plt.tight_layout()
#plt.savefig('figure_chi0_k_bandpath.pdf')